## Pitch-Correction Boxes and Postprocessing

This tool reads songs from /Akamai/voices/data/pitches-vuv-new. In this directory the Crepe pitch estimates are manually corrected.

This tool is used to create pitch-correction boxes and to use note-detection to postprocess the Crepe pitch estimates.

#### Pitch-Correction Boxes

The user creates one or more pitch-correction boxes for each voice of each song. The tool writes the pitch-correction boxes to /Akamai/voices/data/pitch-corrections.

Each pitch-correction box defines a min and max pitch in a range of time steps, as guidance for the other algorithms.

#### Postprocessing

Post-processing sets to -1 all pitches that are not assigned to notes or inter-note runs. 

#### Usage

On each song, the user should
1. Run the manual correction tool separately on each voice, to check they're ready for pitch-correction and postprocessing
2. Make pitch-correction boxes for all three voices.
3. Postprocess all three voices.

The postprocessed pitch estimates are saved to /Akamai/voices/data/pitches-postprocessed.

In [ ]:
import plotly.graph_objs as go
import plotly.offline as py
from plotly.subplots import make_subplots
import pandas as pd
import os
import plotly.io as pio
from plotly.offline import init_notebook_mode, iplot, plot
import math
import copy
import numpy as np
from IPython.display import display, clear_output
from ipywidgets import widgets, Button, HBox, VBox
from plotly.colors import DEFAULT_PLOTLY_COLORS

import random



In [ ]:
def frequency_to_note(frequency):
    # define constants that control the algorithm
    NOTES = ['C', 'C#', 'D', 'D#', 'E', 'F', 'F#', 'G', 'G#', 'A', 'A#', 'B'] # these are the 12 notes in each octave
    OCTAVE_MULTIPLIER = 2 # going up an octave multiplies by 2
    KNOWN_NOTE_NAME, KNOWN_NOTE_OCTAVE, KNOWN_NOTE_FREQUENCY = ('A', 4, 440) # A4 = 440 Hz

    # calculate the distance to the known note
    # since notes are spread evenly, going up a note will multiply by a constant
    # so we can use log to know how many times a frequency was multiplied to get from the known note to our note
    # this will give a positive integer value for notes higher than the known note, and a negative value for notes lower than it (and zero for the same note)
    note_multiplier = OCTAVE_MULTIPLIER**(1/len(NOTES))
    frequency_relative_to_known_note = frequency / KNOWN_NOTE_FREQUENCY
    distance_from_known_note = math.log(frequency_relative_to_known_note, note_multiplier)

    # round to make up for floating point inaccuracies
    distance_from_known_note = round(distance_from_known_note)

    # using the distance in notes and the octave and name of the known note,
    # we can calculate the octave and name of our note
    # NOTE: the "absolute index" doesn't have any actual meaning, since it doesn't care what its zero point is. it is just useful for calculation
    known_note_index_in_octave = NOTES.index(KNOWN_NOTE_NAME)
    known_note_absolute_index = KNOWN_NOTE_OCTAVE * len(NOTES) + known_note_index_in_octave
    note_absolute_index = known_note_absolute_index + distance_from_known_note
    note_octave, note_index_in_octave = note_absolute_index // len(NOTES), note_absolute_index % len(NOTES)
    note_name = NOTES[note_index_in_octave]
    return note_name

#### Note class

In [ ]:
debug = True

# Encapsulates a contiguous sequence of samples of similar pitches 
class Note:
    def __init__(self, sumFreq, numPitches):
        self.minSize = 7
        self.sumFreq = sumFreq
        self.numPitches = numPitches
        
    def update(self, freq, i):
        self.sumFreq += freq[i]
        self.numPitches += 1
    
    # Called after the last pitch of the Note is seen
    def set(self, freq, y, i, riseOrFall, setROF):
        lastNoteSet = False
        if debug:
            print(f"Setting note of length {self.numPitches} before time {i}")
        if self.numPitches >= self.minSize:
            if setROF:
                self.setRiseOrFall(freq, y, riseOrFall)
            else:
                self.discardPitches(freq, riseOrFall)
            self.setNote(freq, y, range(i-self.numPitches, i))
            lastNoteSet = True 
        else:
            self.discardPitches(freq, range(riseOrFall.start, i))
        self.reset(freq, i)
        return lastNoteSet
        
    def setRiseOrFall(self, freq, y, riseOrFall):
        if debug:
            print(f"Range ({riseOrFall.start}, {riseOrFall.stop}): rise or fall.")
        for i in riseOrFall:
            freq[i] = y[i]/2
    
    def setNote(self, freq, y, rng):
        frequency = self.sumFreq/self.numPitches
        if debug:
            print(f"Range ({rng.start}, {rng.stop}): {frequency}.")
        for i in rng:
            freq[i] = frequency
            
    def discardPitches(self, freq, rng):
        if debug:
            print(f"Range ({rng.start}, {rng.stop}): discarded.")
        for i in rng:
            freq[i] = 0
            
    def reset(self, freq, i):
        self.sumFreq = 0
        self.numPitches = 0
        
    


### Post-Processing utilities

In [ ]:
def isolated0s(a):
    return np.r_[False, (a[:-2] > 0) & (a[1:-1] == 0) &  (a[2:] > 0), False]

def isolated00s(a):
    first0s = (np.r_[False, False, (a[:-5] > 0) & (a[1:-4] > 0) & (a[2:-3] == 0)
              & (a[3:-2] == 0) & (a[4:-1] > 0) & (a[5:] > 0), False, False, False])
    return np.r_[False, first0s[1:] | first0s[:-1]]

# replace isolated 0's in z with notes in zref, and put the result into w
def fix0s(w, z, zref):
    np.putmask(w, isolated0s(z), zref)
    np.putmask(w, isolated00s(z), zref)


tolerance1 = 0.07
tolerance2 = 0.05
def inNbhd(p, q,tolerance = tolerance2):
    return (np.abs(p-q) <= (p*tolerance if p<q else q*tolerance))

### Pitches to Notes - Two versions

In [ ]:
# define "notes" and store them in freq:
# remove isolated 0's from y
# - for large enough a, if y[i+j] ~ y[i], j = 1, ..., a-1 but not a,  
#   the values of freq[i], freq[i+1], ..., freq[i+a-1] are set to the average of the y's
#   and i is set to i+a 
# - otherwise:
#   - i is saved to the current run of bad pitches, and i is set to i+1
#   - when the run ends, set freq[i] for each i in the run, to:
#     - freq[i]/2, if the run is monotone
#     - 0, otherwise
def setRun(frq, st, end):
    diffs = np.ediff1d(frq[max(st-1, 0):end+1]) # next - current
    if len(diffs) > 0 and (np.max(diffs) <= 0 or np.min(diffs) >= 0):
        for q in range(st, end):
            frq[q] = frq[q]/2
    else:
        for q in range(st, end):
            frq[q] = 0

def pitchesToNotes1(y, yref):
    yfix = np.array(y)
    fix0s(yfix, y, yref)
    diffs = np.ediff1d(yfix) # next - current
    isMax = np.r_[False, (diffs[1:]<0) & (diffs[:-1] >=0), False]
    isMin = np.r_[False, (diffs[:-1]<0) & (diffs[1:]>=0), False]
    freq = np.array(yfix).flatten()
    i = 0
    st = 0
    while i < len(freq):
        refFreq = freq[i]
        sumFreq = freq[i]
        a = 1
        while i+a < len(freq) and inNbhd(refFreq, freq[i+a], tolerance1):
            sumFreq += freq[i+a]
            if (isMin[i+a] and (freq[i+a] > refFreq)) or (isMax[i+a] and (freq[i+a] < refFreq)):
                refFreq = freq[i+a]
            a += 1

        if a > 6:
            for q in range(i,i+a):
                freq[q] = sumFreq/a
            setRun(freq, st, i)
            i += a
            st = i
        else:
            i += 1  
    return freq

In [ ]:
# define Notes and store them in freq: 
# remove isolated 0's from y
# as i is incremented,
# - y[i] is in the "neighborhood" of the nearest local min or max if it's within tolerance of their pitches
#   - if its in both neighborhoods the neighborhoods merge
#   - if its in neither, the previous neighborhood is complete and i belongs to a "rising or falling tone"
# - once a neighborhood is is complete:
#   - if it contains at least noteSize pitches it's a note:
#     freq[j] = avg of pitches in a note, for j in the note
#   - freq[j] = 0, otherwise
# - once y[i] is again in the neighborhood of a local min or max, 
#   the rising or falling tone is complete:
#   - if it is followed by a note:
#     freq[j] = y[j]/2, for j in the rising or falling tone
#   - freq[j] = 0, otherwise
#
# For now, reuses one Note object
def pitchesToNotes2(y, yref):
    
    # get local maxes and mins, etc.
    yfix = np.array(y)
    fix0s(yfix, y, yref)
    diffs = np.ediff1d(yfix) # next - current
    isMax = np.r_[False, (diffs[1:]<0) & (diffs[:-1] >=0), False]
    isMin = np.r_[False, (diffs[:-1]<0) & (diffs[1:]>=0), False]
    isMinOrMax = isMin | isMax
    diffs = np.append(diffs, [diffs[len(diffs)-1]])
    argMinOrMax = np.nonzero(isMinOrMax)[0]
    minOrMax = yfix[isMinOrMax]
    print("Min and Max Indices: ", argMinOrMax[5:15])
    print("Mins and Maxes: ", minOrMax[5:15])
    assert(y.shape == diffs.shape and y.shape == isMinOrMax.shape)
    freq = np.array(yfix).flatten()
    i = 0
    m = 0    
    # rise or fall to within tolerance of first local min or max
    while not inNbhd(yfix[i], minOrMax[m]):
        i += 1
    riseOrFall = range(i)
    
    # while i is less than the first local min or max
    note = Note(0, 0)
    while i < argMinOrMax[m]:
        note.update(freq, i) 
        i += 1    
    m += 1
    
    # while i is >= one local min or max and < the next,
    # state transtitions: InLastNbhd -> [[InBothNbhds | InRiseOrFall] -> InNextNbhd -> InLastNbhd
    #                                    | InNextNbhd -> InLastNbhd
    #                                    | InLastNbhd]
    state = "InLastNbhd"
    riseOrFallStart = 0
    lastNoteSet = False
    while m < len(minOrMax):
        print(f"i: {i}, m: {m}, last minOrMax: {minOrMax[m-1]}, next minOrMax: {minOrMax[m]}")
        while i < argMinOrMax[m]:
            # switch statement -- no two cases coexist, and in every case i->i+1
            # when we enter the statement, we are in the state at time i-1. 
            # we must transit to the state at time i
            # if a note ends at time i-1: at time i, call note.set(), which resets note
            # note.update() is called at the end of each case; the value is never used in state InRiseOrFall
            if state == "InLastNbhd":
                if inNbhd(yfix[i], minOrMax[m-1]):
                    if i == argMinOrMax[m-1]: # new note, no riseOrFall
                        lastNoteSet = note.set(freq, yfix, i, riseOrFall, lastNoteSet)
                        riseOrFall = range(i, i)                     
                    if inNbhd(yfix[i], minOrMax[m]):
                        state = "InBothNbhds" 
                elif inNbhd(yfix[i], minOrMax[m]): # new note, no riseOrFall
                    state = "InNextNbhd" 
                    lastNoteSet = note.set(freq, yfix, i, riseOrFall, lastNoteSet)
                    riseOrFall = range(i, i) 
                else:
                    state = "InRiseOrFall"
                    riseOrFallStart = i
                    lastNoteSet = note.set(freq, yfix, i, riseOrFall, lastNoteSet)
            elif state == "InBothNbhds":
                if i == argMinOrMax[m-1] and not inNbhd(yfix[i], minOrMax[m]):
                    state = "InLastNbhd"
                elif not inNbhd(yfix[i], minOrMax[m-1]):
                    state = "InNextNbhd"
            elif state == "InRiseOrFall":
                if i == argMinOrMax[m-1]: # new note, end of riseOrFall
                    note.reset(freq, i)
                    riseOrFall = range(riseOrFallStart, i+1)
                    state = "InLastNbhd" if not inNbhd(yfix[i], minOrMax[m]) else "InBothNbhds"
                elif inNbhd(yfix[i], minOrMax[m]): # new note, end of riseOrFall
                    state = "InNextNbhd"
                    riseOrFall = range(riseOrFallStart, i)
                    note.reset(freq, i)
            else: # InNextNbhd  
                    if i == argMinOrMax[m-1]:
                        state = "InLastNbhd" if not inNbhd(yfix[i], minOrMax[m]) else "InBothNbhds"   
            note.update(freq, i) 
            if debug and i in range(8480, 8500):
                print(f"i: {i}, yfix: {yfix[i]}, State: {state}, riseOrFall: {riseOrFall}, Num pitches: {note.numPitches}")
            i += 1 
        m += 1
    
    # e.g., drop to 0 in mid-note 
    if state == "InLastNbhd":
        lastNoteSet = note.set(freq, yfix, i, riseOrFall, lastNoteSet)
        
    # last rise or fall, typically final 0's
    note.setRiseOrFall(freq, yfix, range(i, len(freq)))
    return freq

pitchesToNotes = pitchesToNotes1

### Collections and Songs

In [ ]:
collections = {"sm":"Scherbaum Mshavanadze",
               "guria":"Teach Yourself Gurian Songs",
               "megrelia":"Teach Yourself Megrelian Songs"}

collection_directories = {"sm":
                          ["GVM009_BatonebisNanina_Tbilisi_Mzetamze_20160919",
                           "GVM017_ChvenMshvidobaTake2_Ozurgeti_ShalvaChemo2016_20160713",
                           "GVM019_DaleKojas_DidgoriVillage_Didgori_20160707",
                           "GVM031_EliaLrde_LakhushdiVillage_MuradGigoGivi_20160819",
                           "GVM097_KristeAghsdga_LakhushdiVillage_MuradGigoGivi_20160819"],
                          "guria":
                          ["Adila-Alipasha",
                           "Indi-Mindi",
                           'Mival Guriashi (1)' ,
                           'Pikris Simghera',
                           "Alaverdi",
                           "K'alos Khelkhvavi",
                           'Mival Guriashi (2)' , 
                           "Sabodisho",
                           "Khasanbegura",     
                           "Mok'le Mravalzhamieri",
                           'Sadats Vshobilvar',
                           "Beri Ak'vans Epareba", 
                           "Lat'aris Simghera",    
                           "Mts'vanesa Da Ukudosa", 
                           "Shermanduli",
                           "Brevalo",             
                           "Manana",         
                           'Nanina (1)',      
                           "Shvidk'atsa",
                           "Chven-Mshvidoba",    
                           "Maq'ruli",               
                           'Nanina (2)',          
                           'Supris Khelkhvavi',
                           'Didi Khnidan',     
                           "Masp'indzelsa Mkhiarulsa", 
                           "Orira",                
                           "Ts'amok'ruli",
                           "Gakhsovs, T'urpa",
                           "Me-Rustveli",        
                           "P'at'ara Saq'varelo"]}

coll_key="guria"
directories = collection_directories[coll_key]

data_dir = "/Akamai/Voice/data/pitches-vuv-new/"
ref_data_dir = "/Akamai/Voice/data/pitches/"
postprocessed_data_dir = "/Akamai/Voice/data/pitches-postprocessed/"
working_coll = collections[coll_key]
larynx = False
working_song = "Alaverdi"

### Load songs into dictionary

In [ ]:
algos = ['boersma', 'noll', 'crepe', 'maddox', 'hermes', 'yin'] # Note: later add praat
data3 = {}
locations = {}
destinations = {}

for algo in algos:
    data3[algo] = {}
    locations[algo] = {}
    destinations[algo] = {}
    for direct in directories:
        data3[algo][direct] = {
            #"mix": {},
            "bass": {},
            "middle": {},
            "top": {}
        }
        locations[algo][direct] = {
            #"mix": {},
            "bass": {},
            "middle": {},
            "top": {}
        }
        destinations[algo][direct] = {
            #"mix": {},
            "bass": {},
            "middle": {},
            "top": {}
        }
        
def separate(adir):
    conv={}
    conv[0] = lambda s: float(s.strip() or 0)
    x,y = np.loadtxt(adir, unpack=True, usecols=(0,1), converters=conv)
    return (x,y)
    
def load_songs():
    global data_dir, working_coll, working_song
    for algorithm in sorted(os.listdir(data_dir)):
        if algorithm in data3:
            print(algorithm)
            for collection in sorted(os.listdir(f"{data_dir}{algorithm}")):
                if collection != working_coll:
                    continue
                print(" ", collection)
                sm = (collection == "Scherbaum Mshavanadze")
                suffix = 'ALRX' if larynx and sm else 'AHDS' 
                for song in sorted(os.listdir(f"{data_dir}{algorithm}/{collection}")):
                    if (song != working_song):
                        continue
                    print("  ", song)
                    for location in sorted(os.listdir(f"{data_dir}{algorithm}/{collection}/{song}")):
                        if (location[-4:] == '.txt'):
                            x, y = separate(f"{data_dir}{algorithm}/{collection}/{song}/{location}")
                            xref, yref = separate(f"{ref_data_dir}{algorithm}/{collection}/{song}/{location}")
                            if (not np.array_equal(x, xref)):
                                print(f"    {location}: different x arrays before and after v-uv")

                            freq = pitchesToNotes(y, yref)
                            
                            notes = []
                            for i in range(0,len(freq)):
                                if freq[i] > 0:
                                    notes.append(frequency_to_note(freq[i]))
                                else:
                                    notes.append('N/A')

                            if suffix in location: #not suffix in location:
                               # data3[algorithm][song]['mix'] = (x, y, freq, yref)
                               # locations[algorithm][song]['mix'] = f"{data_dir}{algorithm}/{collection}/{song}/{location}"
                            #else:
                                #print(location)
                                name_tag = location[(location.index(suffix) + 3):(location.index(suffix) + 6)]
                                sm_kludge = sm and (not 'Batonebis' in location) # I renamed the Batonebis files :-p
                                if (name_tag[1] == '1' and not sm_kludge) or (name_tag[1] == '3' and sm_kludge):
                                    data3[algorithm][song]['bass'] = (x, y, freq, yref)
                                    locations[algorithm][song]['bass'] = f"{data_dir}{algorithm}/{collection}/{song}/{location}"
                                    destinations[algorithm][song]['bass'] = f"{postprocessed_data_dir}{algorithm}/{collection}/{song}/{location}"

                                elif name_tag[1] == '2':
                                    data3[algorithm][song]['middle'] = (x, y, freq, yref)
                                    locations[algorithm][song]['middle'] = f"{data_dir}{algorithm}/{collection}/{song}/{location}"
                                    destinations[algorithm][song]['middle'] = f"{postprocessed_data_dir}{algorithm}/{collection}/{song}/{location}"
                                else:
                                    data3[algorithm][song]['top'] = (x, y, freq, yref)
                                    locations[algorithm][song]['top'] = f"{data_dir}{algorithm}/{collection}/{song}/{location}"
                                    destinations[algorithm][song]['top'] = f"{postprocessed_data_dir}{algorithm}/{collection}/{song}/{location}"

    print("\nLoaded song data from files into dictionary")
                       
load_songs()                                
                                

### Graph the pitches

- Each algo:voice is a plotly "trace"
- In addition, there is a trace for the calculated "notes" of the target algo
- - This trace can't be selected or saved
- - After altering the target algo, re-graph to get the recalculated notes.

In [ ]:
selectedPoints = {}
targetSong = ""
traceIds = {}
targetAlgorithm = ""
undoState = []
selectionXrange = []
selectionYrange = []
   
def selection_fn(trace,points,selector):
    global targetSong, selectedPoints, selectionXrange, selectionYrange
    selectionXrange = selector.xrange
    selectionYrange = selector.yrange
    splitArray = points.trace_name.split(": ")
    algoName = splitArray[0]
    if algoName == f"{targetAlgorithm}Notes":
        return
    voice = splitArray[1]
    if(len(trace.selectedpoints) != 0):                
        selectedPoints[algoName+"-"+voice] = trace.selectedpoints
    ##print(points)
    #print(trace.selectedpoints)
    
def graph(song, targetAlgo, reload=False):
    global targetSong, targetAlgorithm, undoState
    targetSong = song
    targetAlgorithm = targetAlgo
    print("target algo updated: " + targetAlgorithm)
    if reload:
        load_songs()
    print("plotting data from dictionary")
    traces = []
    type_list = []
    traceId = 0
    for algo, collection in data3.items():
        for name, part in collection.items():
            for audio_type, res in part.items():
                if name != song:
                    continue

                algolabel=algo
                try:
                    trace = go.Scattergl(
                                x = res[0],
                                y = res[1],
                                name=f"{algolabel}: {audio_type}",
                                mode="markers",
                                visible= (True if audio_type == "bass" else False)
                            )

                    traceIds[algolabel+"-"+audio_type] = traceId
                    traceId += 1
                    traces.append(trace)
                    type_list.append(audio_type)
                    
                    if algo == targetAlgorithm:
                        algolabel=algo + "Notes"
                        trace = go.Scattergl(
                                    x = res[0],
                                    y = res[2],
                                    name=f"{algolabel}: {audio_type}",
                                    mode="markers",
                                    visible= (True if audio_type == "bass" else False)
                                )

                        traceIds[algolabel+"-"+audio_type] = traceId
                        traceId += 1
                        traces.append(trace)
                        type_list.append(audio_type)
                        
                except:
                    print(f"{algolabel}: {audio_type} not available")

    layout = go.Layout(title='Activity Heatmap')

    figure = go.Figure(data=traces, layout=layout)

    fig = go.FigureWidget(figure)

    for i in range(0,len(traces)):
        fig.data[i].on_selection(selection_fn)

    buttons = []
    labels = ['bass', 'middle', 'top'] #, 'mix']
    for i, label in enumerate(labels):
        visibility = [label==current_type for current_type in type_list]
        button = dict(
            label = label,
            method = "update",
            args = [{ 'visible': visibility}]
        )

        buttons.append(button)

    updatemenus = list([
        dict(active=0,
            buttons=buttons,
            direction="down",
            pad={"r": 10, "t": 10},
            showactive=True,
            xanchor="left",
            yanchor="top",
            x = 0.005,
            y = 1.06,
        )
    ])

    fig.update_layout(
        legend=dict(
        orientation="h",
        yanchor="bottom",
        y=1.02,
        xanchor="right",
        x=1
        ),
        margin=dict(l=0, r=0, t=100, b=0)
    )


    fig.update_traces(
        marker=dict(size=3),
        selector=dict(mode='markers')
    )


    fig['layout']['title'] = "Fixing " + targetAlgorithm + " for " + song
    fig['layout']['width'] = 900
    fig['layout']['height'] = 500
    fig['layout']['showlegend'] = True
    fig['layout']['updatemenus'] = updatemenus
    
    undoState = copy.deepcopy(fig.data)


    def update_axes(Octave):
        print("Correct octave updated: " + Octave)
        
        
    def on_button_octave(x):
        global targetAlgorithm, targetSong
        undoState = copy.deepcopy(fig.data)
        for algo in selectedPoints:
            splitArray = algo.split("-")
            algoName = splitArray[0]
            voice = splitArray[1]
            if algoName != targetAlgorithm:
                newData = np.array(fig.data[traceIds[targetAlgorithm+"-"+voice]].y)
                if len(selectedPoints[algo]) > 0:
                    selectedpoints = list(selectedPoints[algo])
                    if(len(selectedPoints[algo]) != 0):    
                        i = selectedPoints[algo][0]
                        #print(data3[algoName][targetSong][voice][2])
                        while True:
                            selectedpoints.append(i)
                            if data3[algoName][targetSong][voice][2][i-1] != data3[algoName][targetSong][voice][2][i]:
                                break;
                            i = i-1
                        i = selectedPoints[algo][-1]
                        while True:
                            selectedpoints.append(i)
                            if data3[algoName][targetSong][voice][2][i+1] != data3[algoName][targetSong][voice][2][i]:
                                break;
                            i = i+1
                    selectedPoints[algo] = tuple(selectedpoints)
                    for point in selectedPoints[algo]:
                        if fig.data[traceIds[targetAlgorithm+"-"+voice]].y[point] > 0 and fig.data[traceIds[algo]].y[point] > 0:
                            distances = list()
                            distances.append(abs(fig.data[traceIds[algo]].y[point]-newData[point]))
                            distances.append(abs(fig.data[traceIds[algo]].y[point]-(newData[point]*2)))
                            distances.append(abs(fig.data[traceIds[algo]].y[point]-(newData[point]/2)))
                            distances.append(abs(fig.data[traceIds[algo]].y[point]-(newData[point]*3)))
                            distances.append(abs(fig.data[traceIds[algo]].y[point]-(newData[point]/3)))
                            distances.append(abs(fig.data[traceIds[algo]].y[point]-(newData[point]*4)))
                            distances.append(abs(fig.data[traceIds[algo]].y[point]-(newData[point]/4)))

                            minimum = distances.index(min(distances))
                            if minimum == 0:
                                newData[point] = newData[point]
                            if minimum == 1:
                                newData[point] = newData[point]*2
                            if minimum == 3:
                                newData[point] = newData[point]*3
                            if minimum == 5:
                                newData[point] = newData[point]*4
                            if minimum == 2:
                                newData[point] = newData[point]/2
                            if minimum == 4:
                                newData[point] = newData[point]/3
                            if minimum == 6:
                                newData[point] = newData[point]/4
                            
                fig.data[traceIds[targetAlgorithm+"-"+voice]].y = newData
                
    def on_button_octave_selected(x):
        global targetAlgorithm, targetSong
        undoState = copy.deepcopy(fig.data)
        for algo in selectedPoints:
            splitArray = algo.split("-")
            algoName = splitArray[0]
            voice = splitArray[1]
            if algoName != targetAlgorithm:
                newData = np.array(fig.data[traceIds[targetAlgorithm+"-"+voice]].y)
                if len(selectedPoints[algo]) > 0:
                    for point in selectedPoints[algo]:
                        if fig.data[traceIds[targetAlgorithm+"-"+voice]].y[point] > 0 and fig.data[traceIds[algo]].y[point] > 0:
                            distances = list()
                            distances.append(abs(fig.data[traceIds[algo]].y[point]-newData[point]))
                            distances.append(abs(fig.data[traceIds[algo]].y[point]-(newData[point]*2)))
                            distances.append(abs(fig.data[traceIds[algo]].y[point]-(newData[point]/2)))
                            distances.append(abs(fig.data[traceIds[algo]].y[point]-(newData[point]*3)))
                            distances.append(abs(fig.data[traceIds[algo]].y[point]-(newData[point]/3)))
                            distances.append(abs(fig.data[traceIds[algo]].y[point]-(newData[point]*4)))
                            distances.append(abs(fig.data[traceIds[algo]].y[point]-(newData[point]/4)))

                            minimum = distances.index(min(distances))
                            if minimum == 0:
                                newData[point] = newData[point]
                            if minimum == 1:
                                newData[point] = newData[point]*2
                            if minimum == 3:
                                newData[point] = newData[point]*3
                            if minimum == 5:
                                newData[point] = newData[point]*4
                            if minimum == 2:
                                newData[point] = newData[point]/2
                            if minimum == 4:
                                newData[point] = newData[point]/3
                            if minimum == 6:
                                newData[point] = newData[point]/4
                            
                fig.data[traceIds[targetAlgorithm+"-"+voice]].y = newData

    def on_button_delete(x):
        undoState = copy.deepcopy(fig.data)
        for algo in selectedPoints:
            #splitArray = algo.split("-")
            #algoName = splitArray[0]
            #voice = splitArray[1]
            #location = locations[algoName][targetSong][voice]
            newData = np.array(fig.data[traceIds[algo]].y)
            if len(selectedPoints[algo]) != 0:
                for point in selectedPoints[algo]:
                    newData[point] = 0
            fig.data[traceIds[algo]].y = newData
    
    def on_button_delete_note(x):
        undoState = copy.deepcopy(fig.data)
        for algo in selectedPoints:
            splitArray = algo.split("-")
            algoName = splitArray[0]
            voice = splitArray[1]
            #location = locations[algoName][targetSong][voice]
            newData = np.array(fig.data[traceIds[algo]].y)
            if len(selectedPoints[algo]) != 0:
                selectedpoints = list(selectedPoints[algo])  
                i = selectedPoints[algo][0]
                print(data3[algoName][targetSong][voice][2])
                while True:
                    selectedpoints.append(i)
                    if data3[algoName][targetSong][voice][2][i-1] != data3[algoName][targetSong][voice][2][i]:
                        break;
                    i = i-1
                i = selectedPoints[algo][-1]
                while True:
                    selectedpoints.append(i)
                    if data3[algoName][targetSong][voice][2][i+1] != data3[algoName][targetSong][voice][2][i]:
                        break;
                    i = i+1
                selectedPoints[algo] = tuple(selectedpoints)
                for point in selectedPoints[algo]:
                    newData[point] = 0
                fig.data[traceIds[algo]].y = newData
            
    # use this button only if no other editing has been done
    # this button 
    # - starts with yfix, which is y (not the current trace) with isolated 0's fixed
    # - sets any pitches that don't belong to notes or between-note rises or falls to -1
    def on_button_postprocess(x):        
        global targetSong, targetAlgorithm
        undoState = copy.deepcopy(fig.data)
        print("Target Algorithm post-processing starting...")
        #fig.restyle(fig, 'selectedpoints', null)
        for scatter in fig.data:
            splitArray = scatter.name.split(": ")
            algoName = splitArray[0]
            voice = splitArray[1]
            fullName = algoName+"-"+voice
            if algoName == targetAlgorithm:                
                yfix = np.array(data3[algoName][targetSong][voice][1])
                fix0s(yfix, data3[algoName][targetSong][voice][1], data3[algoName][targetSong][voice][3])
                #newData = np.array(fig.data[traceIds[algoName+"-"+voice]].y)
                frq = data3[algoName][targetSong][voice][2]
                for i in range(0, len(yfix)):
                    if frq[i] == 0 and yfix[i] != 0:
                        yfix[i] = -1                
                fig.data[traceIds[algoName+"-"+voice]].y = yfix
        print("Target Algorithm post-processing done.")
    
    def on_button_save(x):
        global targetSong
        print("Beginning saving files...")
        for scatter in fig.data:
            splitArray = scatter.name.split(": ")
            algoName = splitArray[0]
            voice = splitArray[1]
            if algoName == targetAlgorithm:
                destination = destinations[algoName][targetSong][voice]
                file = open(destination, "w")
                for i in range(0,len(fig.data[traceIds[algoName+"-"+voice]].y)):
                    file.write(str(fig.data[traceIds[algoName+"-"+voice]].x[i])+" "+str(fig.data[traceIds[algoName+"-"+voice]].y[i]))
                    file.write("\n")
                file.close()
                print("Saved", destination)
            
    def on_button_undo(x):
        global undoState
        for scatter in fig.data:
            splitArray = scatter.name.split(": ")
            algoName = splitArray[0]
            voice = splitArray[1]
            fullName = algoName+"-"+voice
            if algoName == "crepe":                
                newData = np.array(undoState[traceIds[algoName+"-"+voice]].y)          
                fig.data[traceIds[algoName+"-"+voice]].y = newData
    
    def on_button_deselect(x):
        global undoState
        for scatter in fig.data:                   
            fig.data[traceIds[algoName+"-"+voice]].selectedpoints = None
            fig.data[traceIds[algoName+"-"+voice]].selectedpoints = []
    
    # inserts box into list of boxes ordered by x coordinates of corners, no two overlapping
    # deals with cases where the new box overlaps a box in the list
    # - no guarantees when the new box overlaps with more than one box in the list
    def on_button_range(x):
        global selectionXrange, selectionYrange
        for scatter in fig.data:
            splitArray = scatter.name.split(": ")
            algoName = splitArray[0]
            if algoName == f"{targetAlgorithm}Notes":
                continue
            voice = splitArray[1]
            location = locations[algoName][targetSong][voice]
            location = location.replace("pitches-vuv-new", "pitch-corrections")
            selLeftX = round(float(selectionXrange[0]),2)
            selRightX = round(float(selectionXrange[1]),2)
            selLeftY = round(float(selectionYrange[0]),2)
            selRightY = round(float(selectionYrange[1]),2)
            if algoName == "crepe" and scatter.visible == True:
                
                with open(location,"r") as f:
                    currentLines = []
                    num_lines = 0
                    allLines = f.readlines()
                    if len(allLines) == 0:
                        print("# first box")
                        currentLines.append(str(selLeftX)+" "+str(selLeftY)+" "+str(selRightX)+" "+str(selRightY))
                    else:
                        print("Inserting ["+str(selLeftX)+" "+str(selLeftY)+" "+str(selRightX)+" "+str(selRightY)+
                              "] into list of "+str(len(allLines))+" boxes")
                        for i in range(0, len(allLines)):
                            num_lines = i+1
                            line = allLines[i]
                            line = line.replace("\n","")
                            lineArray = line.split(" ")
                            [lineLeftX, lineLeftY, lineRightX, lineRightY] = [float(numeric_string) for numeric_string in lineArray]
                                
                            if lineLeftX < selLeftX:
                                print("# new box entirely to the right")
                                currentLines.append(line)
                                if i+1 == len(allLines):
                                    currentLines.append(str(selLeftX)+" "+str(selLeftY)+" "
                                                        +str(selRightX)+" "+str(selRightY))
                                    break
                                # else try next box in list
                            elif (lineLeftX < selLeftX < lineRightX or 
                                lineLeftX < selRightX < lineRightX):
                                print("# new box overlaps old...")
                                if (lineLeftX < selLeftX < lineRightX and 
                                    lineLeftX < selRightX < lineRightX): 
                                    print("## ... in between")
                                    currentLines.append(str(lineLeftX)+" "+str(lineLeftY)+" "
                                                        +str(selLeftX)+" "+str(lineRightY))
                                    currentLines.append(str(selLeftX)+" "+str(selLeftY)+" "
                                                        +str(selRightX)+" "+str(selRightY))
                                    currentLines.append(str(selRightX)+" "+str(lineLeftY)+" "
                                                        +str(lineRightX)+" "+str(lineRightY))
                                    break
                                elif lineLeftX < selLeftX < lineRightX:
                                    print("## ... on the right")
                                    currentLines.append(str(lineLeftX)+" "+str(lineLeftY)+" "
                                                        +str(selLeftX)+" "+str(lineRightY))
                                    currentLines.append(str(selLeftX)+" "+str(selLeftY)+" "
                                                        +str(selRightX)+" "+str(selRightY))
                                    break
                                elif lineLeftX < selRightX < lineRightX:
                                    print("## ... on the left")
                                    currentLines.append(str(selLeftX)+" "+str(selLeftY)+" "
                                                        +str(selRightX)+" "+str(selRightY))
                                    currentLines.append(str(selRightX)+" "+str(lineLeftY)+" "
                                                        +str(lineRightX)+" "+str(lineRightY))
                                    break
                            elif selLeftX < lineLeftX and selRightX > lineLeftY:
                                print("# new box covers old")
                                if i+1 < len(allLines):
                                    nextLine = allLines[i+1]
                                    nextLine = nextLine.replace("\n","")
                                    nextLineArray = nextLine.split(" ")
                                    [nlLeftX, d1, d2, d3] = [float(numeric_string) for numeric_string in nextLineArray]
                                    
                                    if nlLeftX > selRightX:
                                        currentLines.append(str(selLeftX)+" "+str(selLeftY)+" "
                                                            +str(selRightX)+" "+str(selRightY))
                                        break
                                    # else continue; new box may overlap the next box, too
                                else:
                                    currentLines.append(str(selLeftX)+" "+str(selLeftY)+" "
                                                        +str(selRightX)+" "+str(selRightY))
                                    break
                            elif lineLeftX > selRightX:
                                print("new box entirely to the left")
                                currentLines.append(str(selLeftX)+" "+str(selLeftY)+" "
                                                    +str(selRightX)+" "+str(selRightY))
                                currentLines.append(line)
                                break
                                    
                        for i in range (num_lines+1,len(allLines)):
                            # add rest of them.
                            line = allLines[i]
                            line = line.replace("\n","")
                            currentLines.append(line)

                    
                           
                with open(location,"w") as f:
                    print(currentLines)
                    for line in currentLines:
                        if(line != ""):
                            #print(line)
                            f.write(line)
                            f.write("\n")
                    
                print("Saved Range", location)
            
    button_postprocess = Button(description=f"Post-process {targetAlgorithm}", button_style='danger')
    #button_octave = Button(description="Correct note", button_style='danger')
    #button_octave_selected = Button(description="Correct selected", button_style='danger')
    #button_delete = Button(description="Delete selected", button_style='danger')
    #button_delete_note = Button(description="Delete note", button_style='danger')
    button_save = Button(description=f"Save {targetAlgorithm}", button_style='danger')
    button_undo = Button(description="Undo", button_style='danger')
    button_deselect = Button(description="Deselect", button_style='danger')
    button_range = Button(description="Set Range", button_style='danger')
    button_postprocess.on_click(on_button_postprocess)
    #button_octave.on_click(on_button_octave)
    #button_octave_selected.on_click(on_button_octave_selected)
    #button_delete.on_click(on_button_delete)
    #button_delete_note.on_click(on_button_delete_note)
    button_save.on_click(on_button_save)
    button_undo.on_click(on_button_undo)
    button_deselect.on_click(on_button_deselect)
    button_range.on_click(on_button_range)

    #button_next.observe(on_button_next, )

    #display(widgets.VBox([fig,HBox([button_postprocess, button_delete, button_delete_note, button_octave_selected, button_octave]), HBox([button_save, button_undo, button_deselect, button_range])]))
    display(widgets.VBox([fig,HBox([button_postprocess, button_save, button_undo, button_deselect, button_range])]))
    

### Can Correct

- Crepe (target algo) notes that should be 0 (usually another voice is singing)
- Crepe (target algo) notes for which another algorithm has a better estimate (in another octave)

### Cannot Correct

- consonants that show up as 0's or other notes (usually no algorithm has a good estimate)
- notes in the wrong octave but for which no algorithm has a good estimate in the right octave
- notes in the right octave that sound wrong 



## Warnings on Use of Tool

- Post-Process button post-processes all three voices
- Save button saves all three voices

Advice: Make pitch-correction boxes for all three voices before postprocessing.

In [ ]:
graph(working_song,'crepe', False)

#display(VBox([button_delete,button_next]))

## Problem sections

### Problems:

- partly-corrected notes
- 0's (probably from Voiced-Unvoiced algorithm) that should be notes

### Non-problems:

- consonants that show up as 0's or other notes
- notes that sound wrong but for which no algorithm offers a correction

#### Syntax

- start_time, algorithm, problem
- first_start_time-last_start_time, algorithm, problem (if there are 2 or more sections in succession)

### Gurian songs

#### Mok'le Mravalzhamieri

Bass

Middle - crosses top a lot

Top

- 3.2, 4.0, 9.5, 10.1, 11.6, 17.4, 19.7 crepe estimates 4th or 5th too high (Noll mostly correct) 
- 18.7 crepe estimate 5th too high, miscorrected (Noll)

#### Mts'vanesa Da Ukudosa

Bass

- 43.9-45.9, crepe 0
- 87.8-88.4, crepe 0
- 129.4-129.8, crepe 0

Middle - corrected consonants when top voice present

Top - corrected consonants when bottom voice present

- 15.6, crepe bass note
- 29.0, 32.2, 39.5, 72.4, 82.4, 102.8, 110.8, 113.5, 123.1, 126.3 crepe deleted mystery estimates during apparent silence
- 70.4, crepe bass note
- 83.5, crepe bass note (top note one octave up, all algorithms pick the bass note)
- 102.1, crepe bass note

#### Nanina (1)

(Lost notes to git?)

#### Nanina (2)

Bass

Middle

Top

- 56.0, crepe estimate is another voice
- 66.7, crepe miscorrected short note to 0

#### Orira

Bass

Middle - hard to tell when the estimate is a bass note throughout; Noll better in some passages

Top - some estimates are other voices: 21.3, 46.0, 63.6, 77.7 (no better estimate)

- 87.3, crepe corrected other voice to a note above Boersma; didn't correct 87.8, 88.4

#### P'at'ara Saq'varelo

Bass

Middle - hard to tell when the estimate is a bass note throughout

- 68.4, crepe consonant changed to 0 because estimate is bass note

Top 

- 

#### Pikris Simghera

Bass

Middle

Top

#### Sabodisho

Bass

- 106.7-107.3, crepe 1 octave low

Middle

- 86.3, crepe 1 octave low
- 92.9-93.7 crepe 0
- 105.3, crepe 0
- 116.1-116.6, crepe 0
- 128.7-130.2, crepe 0
- 148.5-149.8, crepe 0
- 168.3-169.3, crepe 0
- 194.2-197.6, crepe 0

Top

### Megrelian Songs

In [ ]:
import glob
for file in glob.iglob("/Users/kutlay/Documents/GitHub/Voice/data/pitches-crepe-range" + '/**/*.txt', recursive=True):
    file = open(file, "w")
    file.write("")
    file.close()

In [ ]:
[lx, ly] = [1, 2]